In [45]:
import pyspark
pyspark.__version__

'3.2.1'

In [46]:
import findspark
findspark.init()
findspark.find()
from pyspark.sql import SparkSession
from pyspark.ml.fpm import FPGrowth
import random
import os

In [47]:
#Classe session pour demarer une nouvelle session spark 
#il a une seul fonction débutSession()
class Session:
    def __init__(self):
        self.spark= SparkSession.builder.appName('SparkByExamples.com').enableHiveSupport().getOrCreate()
    def debutSession(self):
        return self.spark



In [48]:

#classe donné pour la génération de jeux de donnée a étudié et chargement dans une Dataframe
#il a 2 fonction dataframe() et affiche()
class Donne:
    liste=[]
    def __init__(self,nbrl,nbre,pas,plage):
        self.nbrl=nbrl
        self.nbre=nbre
        self.pas=pas
        self.plage=plage
        self.liste=[]
        for i in range(0, self.nbrl):
            random1 = random.sample(range(self.plage), self.nbre)
            self.liste.append((random1,))
            if (self.nbre<= self.pas):
                self.nrbe =0
            else:
                self.nbre=self.nbre-self.pas
    
    def dataframe(self):
        return spark.createDataFrame(self.liste, ["items"])
    
    def affiche(self):
        print(self.liste)

In [49]:
# classe Traitement qui utulise les données générer pour les traité et construire une modéle
#il a 2 fonction modéle() et save_model()
class Traitement:
    def __init__(self,donne):
        self.donne=donne
    def modele(self):
        fpGrowth = FPGrowth(itemsCol="items", minSupport=0.5, minConfidence=0.6)
        return fpGrowth.fit(self.donne)
    
    def save_model(self,model):
        path="/"
        if not os.path.exists(path):
            print('save directories...', flush=True)
            os.makedirs(path)
        path =path + 'model'
        model.save(path)
        
        return path

In [50]:
#utilisation de la classe session() pour débuter une nouvelle session 
sess = Session()  
spark=sess.debutSession()
spark

In [51]:
#génération de données avec la classe Donne(nbrl,nbre,pas,plage) il prend en argument nbrl=nombre de liste dans le jeu 
#de donné ,nbre=nombre d'élément de la premiére liste et peut étre aussi des listes suivant,
# pas= qui permet de voir la variation du nombre d'élément dans les listes 
#et le plage= qui donne la plage de valeur qu'on peut générer de 0 au valeur désigné
lis=Donne(20,40,1,50)
test=Donne(10,40,1,50)
#fonction affiche qui permet d'affiché le jeux de donnée
lis.affiche()
#fonction dataframe il permet de charger le jeux de donnée généré en dataframe spark
df = lis.dataframe()
df.show()
dfe= test.dataframe()
dfe.show()


[([36, 32, 43, 26, 45, 48, 35, 38, 10, 16, 19, 25, 5, 15, 37, 39, 14, 20, 4, 23, 49, 11, 1, 2, 21, 27, 30, 34, 18, 42, 9, 46, 28, 47, 3, 7, 31, 13, 6, 8],), ([6, 24, 36, 31, 11, 48, 38, 29, 14, 19, 49, 44, 15, 32, 2, 12, 0, 35, 27, 46, 30, 45, 42, 18, 40, 1, 4, 23, 41, 3, 28, 20, 47, 10, 22, 13, 33, 9, 16],), ([2, 39, 47, 8, 17, 35, 32, 24, 21, 43, 10, 7, 23, 13, 0, 16, 28, 1, 14, 41, 26, 18, 42, 9, 20, 30, 25, 3, 4, 27, 11, 36, 31, 33, 12, 45, 6, 40],), ([45, 12, 13, 33, 40, 24, 19, 47, 31, 38, 44, 41, 39, 23, 3, 35, 26, 7, 37, 30, 34, 2, 27, 43, 46, 18, 36, 1, 28, 0, 20, 14, 9, 16, 11, 5, 49],), ([45, 12, 15, 35, 44, 5, 48, 17, 0, 34, 38, 36, 9, 43, 14, 29, 11, 7, 19, 6, 23, 42, 21, 22, 13, 1, 39, 41, 16, 20, 25, 32, 27, 4, 37, 33],), ([39, 29, 32, 12, 28, 6, 5, 8, 44, 30, 3, 16, 21, 13, 40, 23, 35, 20, 15, 37, 26, 33, 43, 45, 9, 48, 7, 10, 0, 18, 36, 38, 46, 31, 49],), ([39, 47, 5, 26, 14, 21, 37, 34, 13, 30, 44, 0, 17, 42, 32, 19, 38, 18, 3, 10, 40, 31, 24, 8, 36, 2, 6, 49, 43, 48,

In [52]:
#la classe Traitement() il prend comme paramétre le dataframe créer avec la classe Donne()    
don=Traitement(df)
#fonction modéle permet de faire l'aprantisage avec FP-Growth il returne un model qu'on va utiliser pour voir les régle association  
model = don.modele()
FPGrowthModel=model.setPredictionCol("newPrediction")

# on a ici une fontion de Fp-growth pour sortir les éléments fréquent de notre model avec le nombre de foi qu'il sorte.
model.freqItemsets.show()

# la fonction associationRules comme son nom l'indique sort les différents régle d'association dans le model.
model.associationRules.show()
#la fonction transform permet d'appliquer le model sur un jeux de donné de text qu'on peut aussi créer avecla classe Donné() 
model.transform(df).show()

+------------+----+
|       items|freq|
+------------+----+
|        [35]|  12|
|     [35, 3]|  11|
|    [35, 14]|  10|
|    [35, 18]|  10|
| [35, 18, 3]|  10|
|    [35, 10]|  10|
| [35, 10, 3]|  10|
|     [35, 1]|  10|
|    [35, 45]|  11|
| [35, 45, 3]|  10|
|[35, 45, 36]|  10|
|    [35, 36]|  10|
|    [35, 20]|  10|
|    [35, 39]|  10|
|         [7]|  14|
|      [7, 3]|  10|
|     [7, 18]|  10|
|      [7, 1]|  10|
|  [7, 1, 13]|  10|
|     [7, 33]|  10|
+------------+----+
only showing top 20 rows

+----------+----------+------------------+------------------+-------+
|antecedent|consequent|        confidence|              lift|support|
+----------+----------+------------------+------------------+-------+
|  [32, 13]|      [42]|0.8333333333333334|1.2820512820512822|    0.5|
|  [32, 13]|       [1]|0.9166666666666666|1.1458333333333333|   0.55|
|  [45, 13]|      [23]|               1.0|1.6666666666666667|    0.5|
|   [1, 13]|       [7]|0.7692307692307693| 1.098901098901099|    0.5|
|   

In [53]:
#save_model est une fonction de Traitement() pour sauvegarder le modéle pour pouvoir l'utiliser une autre fois
model_path=don.save_model(model)


In [54]:
#on utilise FPGROWTHModel pour charger le modéle enregistré
model2 = FPGrowthModel.load(model_path)
#cette instrution permet de vérifié estce que le modéle a été bien enregistré il renvoi true si c le cas
#et false si le modéle n'est pas bien enregistré 
model.transform(df).take(1) == model2.transform(df).take(1)

False